In [254]:
%reload_ext autoreload
%autoreload 2

In [255]:
from test_extract_file_lists import test_get_config
import numpy as np
import pandas as pd
from pandas_summary import DataFrameSummary
from toolz import concatv, first
from pathlib import Path
from nbc_analysis.utils.debug_utils import runit
from nbc_analysis.utils.file_utils import init_dir

from test_extract_events import test_extract_events

%matplotlib inline

In [256]:
CONFIG_F = 'default'

config= runit(test_get_config)
config
BATCHES_D  = Path(config['BATCHES_D'])

>> created example config file '/Users/wmcabee/.config/nbc_analysis/config_example.yaml'
>> Using default config


In [257]:
init_dir(BATCHES_D, rmtree=True)
ret = runit(test_extract_events)


>> created example config file '/Users/wmcabee/.config/nbc_analysis/config_example.yaml'
>> Using default config
>> start extract events, config={'VIDEO_END_BUCKET': 'nbc-event', 'BATCHES_D': '/Users/wmcabee/DATA/NBC2/batches', 'FILE_LISTS_D': '/Users/wmcabee/DATA/NBC2/file_lists', 'BATCH_SPEC_D': '/Users/wmcabee/DATA/NBC2/batch_spec', 'PARTITIONS_D': '/Users/wmcabee/DATA/NBC2/partitions', 'BATCH_SIZE': 200000000, 'DAYS_LIMIT': 7, 'BATCH_LIMIT': 2, 'BATCH_FILES_LIMIT': 2}
>> WARNING: Limit batch count,batch_limit=2
>> WARNING: Limit files in batch to first n,batch_files_limit=2
>> start event download,batch_id=ve_20190701_0000
>> wrote partition /Users/wmcabee/DATA/NBC2/batches/ve_20190701_0000.parquet.gz,row_cnt=385
>> start event download,batch_id=ve_20190701_0001
>> wrote partition /Users/wmcabee/DATA/NBC2/batches/ve_20190701_0001.parquet.gz,row_cnt=734
>> end start extracts


In [252]:
indir  = Path(config['BATCHES_D'])
reader = indir.glob('*.parquet.gz')
reader = map(pd.read_parquet, reader)
df = pd.concat(reader)

In [253]:
dfs = DataFrameSummary(df)
dfs.columns_stats.T

,counts,uniques,missing,missing_perc,types
batch_id,32536,2,0,0%,bool
file,32536,200,0,0%,categorical
file_idx,32536,200,0,0%,numeric
asof_dt,32536,2,0,0%,bool
mpid,32536,18292,0,0%,numeric
nbc_profile,32536,3,0,0%,categorical
mvpd,32536,185,0,0%,categorical
event_name,32536,1,0,0%,constant
event_type,32536,1,0,0%,constant
platform,32536,5,0,0%,categorical


In [231]:

# video_duration_watched has large negative nubers
# session_start_unixtime_ms has null values
# IP has 2 different formats
# Some records have null value for platforms
# What is resume

column = 'video_type'
dfs[column]

top             Full Episode: 28013
counts                        32477
uniques                          12
missing                          59
missing_perc                  0.18%
types                   categorical
Name: video_type, dtype: object

In [232]:
df[column].value_counts(dropna=False).sort_index()

Behind the Scenes        7
Current Preview        271
Excerpt                 34
Full Episode         28013
Highlight             2910
Interview              165
Movie                  112
Not Set                  1
Sneak Peek             154
Teaser Trailer           1
Trailer                  4
Web Exclusive          805
NaN                     59
Name: video_type, dtype: int64

In [233]:
mask = df[column].astype(np.float) < -40000
df[mask].T

ValueError: could not convert string to float: 'Full Episode'

In [218]:
df[df[column] == '1561939368868'].T

,1090,1091
batch_id,ve_20190701_0000,ve_20190701_0000
file,NBC_App_20190701000700217176_78556.txt,NBC_App_20190701000700217176_78556.txt
file_idx,9,9
asof_dt,2019-10-01T18:29:17.131342Z,2019-10-01T18:29:17.131342Z
mpid,-8018306495628861984,-3754618777285587871
nbc_profile,Unauthenticated,Unauthenticated
mvpd,Dish,Not Set
event_name,Video End,Video End
event_type,custom_event,custom_event
platform,tvOS,tvOS


In [197]:
df.session_id

0        8440387147641549297
1        5384130267361706691
2       -5898373881150763663
3         971402367957401092
4        2582458453689969205
                ...         
1128    -4110787067288868390
1129    -6796986825325535070
1130     1481375352496432888
1131     6616771921888612350
1132     3582251226897962777
Name: session_id, Length: 3423, dtype: object